In [1]:
from scripts.setup_libs import *

# SLIDE (2) X-regularization

Разберемся со сложной бустинговой регулризацией. 
У нас есть реальные значения $y$ и некоторый неполный алгоритм $a$, который выдает по объектам значения $y^a$ ($y_i^{(t-1)}$ в теории)

Мы создаем следующее дерево решений и оно как-то разбивает наши объекты на листья. (Разбение таргетов $Tree$ - содержит индексы объектов, а не сами объекты)

Необходимо подсчитать функционал и новые оптимальные веса объектов.
$$ Q = -\frac{1}{2}\sum_{j=1}^T \frac{G_j^2}{H_j + \lambda} + \gamma T$$

$$ w_j^* = -\frac{G_j}{H_j + \lambda}$$

 - $T$ — количество листьев в текущем дереве
 - $\lambda, \gamma$ — гиперпараметры
 - $G_j = \sum_{i\in I_j}g_i$ - сумма градиентов в одном листе по $g_i = \partial_{\hat{y}_i^a} \mathcal{L}(y_i, \hat{y}_i^a)$
 - $H_j = \sum_{i\in I_j}h_i$ - сумма гессианов в одном листе по $h_i = \partial_{\hat{y}_i^a}^2 \mathcal{L}(y_i, \hat{y}_i^a) $
 - $\mathcal{L}(y_i, \hat{y}_i^a) = (y_i - \hat{y}_i^a)^2$
 - $\hat{y}_i^a$ - значение **уже собранного** алгоритма 



# TASK

In [ ]:
def xregul(y, y_prev, Tree):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    return model

# TESTS

In [3]:
lamb = 0.5
gamma = 0.5
y = np.array([2, 4, 6, 8, 24, 26, 28, 34])
y_prev = np.array([5, 5, 5, 5, 27, 27, 27, 27])
Tree = np.array([np.array([0, 1, 2]), 
                 np.array([3]), 
                 np.array([4, 5, 6]), 
                 np.array([7])])


Q, w = xregul(y, y_prev, Tree, gamma, lamb)
    
assert (Q - -50) < 0.1
assert_almost_equal(w, np.array([-0.92, 2.4, -0.92, 5.6]), 2)

#########################################
lamb = 0.5
gamma = 0.5
y = np.array([2, 4, 6, 8, 24, 26, 28, 34])
y_prev = np.array([3, 3, 7, 7, 26, 26, 26, 34])
Tree = np.array([np.array([0, 1, 2]), 
                 np.array([3]), 
                 np.array([4, 5, 6]), 
                 np.array([7])])


Q, w = xregul(y, y_prev, Tree, gamma, lamb)
    
assert (Q - 0.89) < 0.1
assert_almost_equal(w, np.array([-0.3, 0.8, 0, 0]), 2)
#########################################

# SLIDE (2) X-regression

Необходимо найти наилучшие параметры для XGBRegression, обучить модель и вернуть ее. Данные брать [отсюда](https://yadi.sk/d/g-drGh61KLxRXQ).

Сам гридсерч или нативное исследование необходимо делать вне функции обработки, чтобы не получить TL.

# TASK

In [ ]:
def xreg(X_train, y_train):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    return model

# TESTS

In [5]:
df = pd.read_csv('data/Financial Distress.csv')

X = df.drop('Financial Distress', axis=1)
y = df['Financial Distress']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=17)

xgb_model = xreg(X_train, y_train)
y_pred = xgb_model.predict(X_test)

assert type(xgb_model) == xgb.sklearn.XGBRegressor
assert MSE(y_pred, y_test) < 3

# SLIDE (2) CatFeatures

Обучите модель классификации катбуста на предложенных данных и верните обученную модель. 

Воспользуйтесь встроенной обработкой категориальных признаков. Не забудьте обработать Nan значения.

Скрытых тестов нет, только один [датасет](https://yadi.sk/i/tAkNkgHICbrHsA).

# TASK

In [204]:
def catfeatures(df: pd.DataFrame):
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    return model

# TEST

In [8]:
%%time
df = pd.read_csv('data/flight_delays_train.csv')
df_train = df[:7000]
model = catfeatures(df_train)


assert type(model) == catboost.CatBoostClassifier

df_test = pd.read_csv('data/flight_catfeature_test.csv')
df_test = df_test.drop('Unnamed: 0', axis=1)
X_test = df_test.drop('dep_delayed_15min',axis=1)
y_test = df_test['dep_delayed_15min']

y_pred = model.predict(X_test)
assert accuracy_score(y_test, y_pred) > 0.80 
assert accuracy_score(y_test, y_pred) < 0.87 


CPU times: user 31.5 s, sys: 2.35 s, total: 33.9 s
Wall time: 5.44 s


# SLIDE(2) Stacking

**Стэкинг** - 3-ий способ комбинирования алгоритмов, кроме бэггинга и бустинга. Он не часто используется, но его идея крайне полезная: `обучение на мета-признаках`.

1. Разобъем нашу обучающую выборку на 2 части: базовую и дополнительную.
2. Возьмем $N$ базовых алгоритмов и обучим их на **базовой части** разбив на $N$ фолдов. (Разбили на $N$ частей и обучаем алгоритм на всех частях кроме одной, как на кросс-валидации)
3. Каждым из обученных базовых алгоритмов предскажем значение для **дополнительной** части выборки.
4. Соберем **мета-выборку**, состоящую из предсказаний базовых алгоритмов на **доп выборе**. Пример: пусть для объекта $x_i$ базовые алгоритмы выдали $(y_i^1 = 1, y_i^2 = 0, y_i^3 = 1)$. Тогда признаками объекта в **мета-выборке** будет вектор $1, 0, 1$.
5. Обучим **мета-алгоритм** на **мета-выборке**. И получим готовую модель.
6. Чтобы получить результат на тестовой, теперь нужно сделать предсказания базовыми алгоритмами, собрать **мета-выборку** и сделать предсказания на **мета-алгоритме**.

Реализуйте стекинг классификацию на **деревьях решений**. Валидация проводится на [данном датасете](https://yadi.sk/d/QOAAohDJoDJVtQ)

# TASK

In [221]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier as DTC

class Stacking():
    def __init__(self, n_estimators=5, max_depth=5):
        self.max_depth_ = max_depth
        self.n_estimators_ = n_estimators
        self.estimators_ = [DTC(max_depth=self.max_depth_) for _ in range(self.n_estimators_)]
        self.meta_estimator_ = DTC(max_depth=self.max_depth_)
        
    def fit(self, X: np.array, y: np.array): 
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        return self
        
    def predict(self, X_test) -> np.array:
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        return y_pred

# TESTS

In [10]:
import sklearn
df = pd.read_csv('data/forest_train.csv')

X = df.drop(columns=['Cover_Type', 'Id']).reset_index(drop=True)
y = df['Cover_Type']
X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, train_size=0.3)

model = Stacking(max_depth=10, n_estimators=3).fit(X_train, y_train)

assert type(model.meta_estimator_) == sklearn.tree.DecisionTreeClassifier

y_pred = model.predict(X_test)
y_pred1 = model.estimators_[0].predict(X_test)
y_pred2 = model.estimators_[1].predict(X_test)
y_pred3 = model.estimators_[2].predict(X_test)

assert accuracy_score(y_pred, y_test) > 0.67

assert accuracy_score(y_pred1, y_test) < accuracy_score(y_pred, y_test)
assert accuracy_score(y_pred2, y_test) < accuracy_score(y_pred, y_test)
assert accuracy_score(y_pred3, y_test) < accuracy_score(y_pred, y_test)